In [1]:
import numpy as np
import os
from keras.preprocessing import image
from keras.models import load_model

Using TensorFlow backend.


In [4]:
captcha_word  = '0123456789+-* '

width = 350
height = 80

word_len = 8
word_class = len(captcha_word)

train_dir = 'D:/data/YZM/first_test'    
train_label =  'test_label.txt'    

#生成字符索引
char_indices = dict((c, i) for i,c in enumerate(captcha_word))
indices_char = dict((i, c) for i,c in enumerate(captcha_word))


#验证码字符串转数组
def captcha_to_vec(captcha):    
   
    vector = np.zeros(word_len * word_class)
    for i,ch in enumerate(captcha):
        idex = i * word_class + char_indices[ch]
        vector[idex] = 1
    return vector

#把数组转换回文字
def vec_to_captcha(vec):
    text = []
    vec[vec < 0.5] = 0    
    char_pos = vec.nonzero()[0]
    for i, ch in enumerate(char_pos):
        text.append(captcha_word[ch % word_class])
    return ''.join(text)


In [5]:
image_list = []

for item in os.listdir(train_dir):
    image_list.append(item)

TEST_NUM = len(image_list)

In [6]:
X = np.zeros((TEST_NUM, height, width, 3), dtype = np.uint8)
y = np.zeros((TEST_NUM, word_len * word_class), dtype = np.uint8)

with open(train_label) as file_object:
    p = file_object.readlines()
for i,img in enumerate(image_list):
    img_path = train_dir + "/" + img
    raw_img = image.load_img(img_path, target_size=(height, width))
    X[i] = image.img_to_array(raw_img)
    for j in range(len(p)):
        if img[:4]==p[j][:4]:
            break;
    s = ''
    flage = 0
    for k in p[j]:
        if k=='=':
            break
        else:
            s += k
    n = len(s[5:])
    if n!= 8:
        for m in range(8-n):
            s += ' '
    y[i] = captcha_to_vec(s[5:])



In [8]:
model=load_model('first_model.h5')    

def testCaptcha(index):
    raw_img = X[index]
    true_label = y[index]
    
    X_test = np.zeros((1, height, width, 3), dtype = np.float32)
    X_test[0] = image.img_to_array(raw_img)
    
    
    result = model.predict(X_test)
    
    vex_test = vec_to_captcha(result[0])
    true_test = vec_to_captcha(true_label)
    
    if vex_test !=true_test:
        print("原始：",true_test,"预测:",vex_test.rstrip(),'=',eval(vex_test))
    if vex_test ==true_test:
        return 1
    else:
        return 0
    
    
c=0
for i in range(TEST_NUM):
    if testCaptcha(i)==1:
               c+=1
print(c/TEST_NUM)

1.0
